In [29]:
#chech version of nature
print(qiskit_nature.__version__)
print(pyscf.__version__)


0.6.2
2.3.0


Build an ElectronicStructureProblem

In [2]:
'''PySCFDriver is a class that allows to build a molecluar object in PySCF. We use it to compute the one body and two body integrals.'''
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g", #slater-type orbitals approximated with 3 gaussians
    charge=0, #int chrge of the molecule
    spin=0, #2S where S spin of the molecule
    unit=DistanceUnit.ANGSTROM,
)

In [3]:
# Problem is a ElectronicStructureProblem object
problem = driver.run()
print(problem)

/home/codespace/.python/current/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [ ]:
# The ElectonicStructureProblem object contains the hamiltonian of the molecule, and electronic integrals which the one and two body integrals
hamiltonian = problem.hamiltonian
coefficients = hamiltonian.electronic_integrals #one and two body integrals are 2d and 4d tensors respectively
print(coefficients.alpha)

In [ ]:
#The hamiltonian object contains the second quantized operator (the two summands of the hamiltonian). 
#This is the electronic part of the hamiltonian (nuclear repulsion energy will be added later as a constant, is available as hamiltonian.nuclear_repulsion_energy)
fermionic_op = hamiltonian.second_q_op()
print(fermionic_op)

Mapping the problem to qubit space

In [31]:
'''Jordan-Wigner transformation maps the fermionic operators to unitary operators acting on qubits (https://arxiv.org/abs/2110.12792).
Doing so, we can map the n-body fermionic operator to a circuit of n-qubit gates.
'''
'''EDIT: bug in qiskit_nature.second_q.transforms.qubit_mapper.py called by JordanWignerMapper. Calls to_do method on FermionicOp object, 
which is not implemented, as the note says. Another mapper should be used, e.g. ParityMapper'''

from qiskit_nature.second_q.mappers import ParityMapper
mapper = ParityMapper()

In [ ]:
qubit_jw_op = mapper.map(fermionic_op)
print(qubit_jw_op)

Classical Solver

In [ ]:
'''ow we need to define a solver, an algorithm that will find the ground state of the hamiltonian. 
One example is the NumPyEigensolver, which uses the NumPy sparse eigenvalue solver to compute the eigenvalues. This is a classical algorithm'''
from qiskit.algorithms.eigensolvers import NumPyEigensolver

numpy_solver = NumPyEigensolver(k=1)
numpy_result = numpy_solver.compute_eigenvalues(qubit_jw_op)
print(numpy_result)